In [1]:
from ftsa.protocols.ccsftsa17.client import Client
from ftsa.protocols.ccsftsa17.server import Server

dimension = 100
inputsize = 16 
keysize = 256
threshold = 14
nclients = 20
dropout = 6

Client.set_scenario(dimension, inputsize, keysize, threshold, nclients)
Server.set_scenario(dimension, inputsize, keysize, threshold, nclients)


# Create clients and server

In [2]:
clients = {}
for i in range(nclients):
    idx = i+1
    clients[idx] = Client(idx)

server = Server()

### **Round 0** -- Advertise Keys

In [3]:
# The clients 
allpks = {}
allpkc = {}
for i in range(nclients):
    clients[i+1].new_fl_step()
    user, pks, pkc = clients[i+1].advertise_keys()
    allpks[user] = pks
    allpkc[user] = pkc

In [4]:
# The server
allpks, allpkc = server.advertise_keys(allpks, allpkc)

### **Round 1** -- Share Keys

In [5]:
# The clients
allekshares = {}
for i in range(nclients):
    user, eshares = clients[i+1].share_keys(allpks, allpkc)
    allekshares[user] = eshares

In [6]:
# The server 
allekshares = server.share_keys(allekshares)

### **Round 2** -- Masked Input Collection

In [7]:
# The clients
allY = {}
for i in range(nclients):
    user, Y = clients[i+1].masked_input_collection(allekshares[i+1])
    allY[user] = Y


In [8]:
# drop some clients
nclientsnew = nclients - dropout
allY = {idx:y for idx, y in allY.items() if idx <= nclientsnew }

In [9]:
# The server
U3 = server.masked_input_collection(allY)

### **Round 3** -- Consistency Check
Depriciated

### **Round 4** -- Unmasking

In [10]:
# The clients
allbshares = {}
allkshares = {}
for i in range(nclientsnew):
    user, kshares, bshares = clients[i+1].unmasking(U3)
    allbshares[user] = bshares 
    allkshares[user] = kshares 

In [11]:
# The server
sumX = server.unmasking(allkshares, allbshares)

In [12]:
# Verify the results
summ=clients[1].X
from operator import add
for i in range(1, nclientsnew):
    summ = list(map(add, summ, clients[i+1].X))


print("Verify: ", sumX  == summ )


Verify:  True
